In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
from torch.nn.functional import one_hot

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [12]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 32, 5)
        self.fc1 = nn.Linear(32* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = (self.fc3(F.relu(self.fc2(F.relu(self.fc1(torch.flatten(x,1)))))))
        return x


net = Net()


In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0)

In [14]:
import time

start = time.time()

results = []

for epoch in range(30):  # loop over the dataset multiple times
    count=0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        count=i
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # print statistics

    print(f'[{epoch + 1}, {count + 1:5d}] loss: {running_loss  :.10f}')
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')   
    
    
    end = time.time()
    print(end - start)
    results += [[epoch,end-start,running_loss,100*correct/total]]
    running_loss = 0.0
print('Finished Training')

end = time.time()
print(end - start)

[1,  1563] loss: 3373.9211885929
Accuracy of the network on the 10000 test images: 33.39 %
91.73011946678162
[2,  1563] loss: 2618.3494874239
Accuracy of the network on the 10000 test images: 43.46 %
184.55028676986694
[3,  1563] loss: 2300.2000101805
Accuracy of the network on the 10000 test images: 49.44 %
276.0256202220917
[4,  1563] loss: 2126.8504866362
Accuracy of the network on the 10000 test images: 53.34 %
365.6388747692108
[5,  1563] loss: 1981.0341823697
Accuracy of the network on the 10000 test images: 56.05 %
455.04471039772034
[6,  1563] loss: 1855.1291596293
Accuracy of the network on the 10000 test images: 57.91 %
545.6343142986298
[7,  1563] loss: 1741.1180825233
Accuracy of the network on the 10000 test images: 60.06 %
636.5454580783844
[8,  1563] loss: 1642.2337473035
Accuracy of the network on the 10000 test images: 61.6 %
727.3316640853882
[9,  1563] loss: 1551.7824099660
Accuracy of the network on the 10000 test images: 62.77 %
817.4618580341339
[10,  1563] loss: 